In [ ]:
import os
import inotify_simple
import resource

In [5]:
if not os.path.exists('/tmp/inotify_test'):
	os.mkdir('/tmp/inotify_test')

flags = inotify_simple.flags
watch_flags = flags.MODIFY | flags.ATTRIB | flags.MOVED_FROM | flags.MOVED_TO | flags.CREATE | flags.DELETE | flags.DELETE_SELF | flags.MOVE_SELF | flags.UNMOUNT | flags.Q_OVERFLOW


with inotify_simple.INotify() as inotify:
	wd = inotify.add_watch('/tmp/inotify_test', watch_flags)
	# Now create, delete and modify some files in the directory being monitored:
	os.chdir('/tmp/inotify_test')

	os.system('mkdir foo')
	wd= inotify.add_watch('/tmp/inotify_test/foo', watch_flags)
	os.system('echo hello > test.txt')
	os.system('echo world >> test.txt')
	os.system('mv test.txt test2.txt')
	os.system('mv test2.txt ../outside.txt')
	os.system('echo hello > ../outside2.txt')
	os.system('mv ../outside2.txt test2.txt')
	os.system('mv test2.txt ../outside3.txt')
	os.system('mv ../outside3.txt test3.txt')
	os.system('rm test3.txt')
	os.system('echo hi >> foo/test_rec.txt')
	os.system('mv foo bar')
	os.system('touch bar/test.txt')
	os.system('echo hello > bar/test.txt')
	os.system('mv bar/test.txt bar/test2.txt')
	os.system('sudo mount -t tmpfs tmpfs bar')
	os.system('sudo umount bar')
	os.system('touch bar/test_temp.txt')
	os.system('touch bar')
	os.system('mkdir -p test/testrecursive')
	inotify.rm_watch(wd)
	os.chdir('/tmp')
	# DELETE_SELF on the original directory. # Also generates an IGNORED event
	# indicating the watch was removed.
	os.system('mv inotify_test inotify_test_2')
	os.system('rm -rf inotify_test_2')
	os.system('mkdir inotify_test')
	events = inotify.read()

Place your finger on the fingerprint reader


In [6]:
decodedEvents = [[event.wd,inotify_simple.flags.from_mask(event.mask),event.cookie,event.name] for event in events]
decodedEvents

[[1, [<flags.CREATE: 256>, <flags.ISDIR: 1073741824>], 0, 'foo'],
 [1, [<flags.CREATE: 256>], 0, 'test.txt'],
 [1, [<flags.MODIFY: 2>], 0, 'test.txt'],
 [1, [<flags.MOVED_FROM: 64>], 5717, 'test.txt'],
 [1, [<flags.MOVED_TO: 128>], 5717, 'test2.txt'],
 [1, [<flags.MOVED_FROM: 64>], 5718, 'test2.txt'],
 [1, [<flags.MOVED_TO: 128>], 5719, 'test2.txt'],
 [1, [<flags.MOVED_FROM: 64>], 5720, 'test2.txt'],
 [1, [<flags.MOVED_TO: 128>], 5721, 'test3.txt'],
 [1, [<flags.DELETE: 512>], 0, 'test3.txt'],
 [2, [<flags.CREATE: 256>], 0, 'test_rec.txt'],
 [2, [<flags.MODIFY: 2>], 0, 'test_rec.txt'],
 [1, [<flags.MOVED_FROM: 64>, <flags.ISDIR: 1073741824>], 5722, 'foo'],
 [1, [<flags.MOVED_TO: 128>, <flags.ISDIR: 1073741824>], 5722, 'bar'],
 [2, [<flags.MOVE_SELF: 2048>], 0, ''],
 [2, [<flags.CREATE: 256>], 0, 'test.txt'],
 [2, [<flags.ATTRIB: 4>], 0, 'test.txt'],
 [2, [<flags.MODIFY: 2>], 0, 'test.txt'],
 [2, [<flags.MOVED_FROM: 64>], 5723, 'test.txt'],
 [2, [<flags.MOVED_TO: 128>], 5723, 'test2.txt

In [7]:
events[0]

Event(wd=1, mask=1073742080, cookie=0, name='foo')

In [8]:
inotify_simple.flags.from_mask(events[-1].mask)[0]

<flags.IGNORED: 32768>

In [9]:
selfDestructMask = flags.UNMOUNT | flags.IGNORED | flags.DELETE_SELF | flags.MOVE_SELF

In [10]:
events[-1].mask & selfDestructMask

32768

In [11]:
from collections import namedtuple
# if type is dir, event can be create 
#  ( need to recursively copy everything as inotifywait does not caputre mkdir -p properly for all sub folders),
#  ,delete, attrib, move
# if type is file, event can be delete, change, attrib, move
# move detection will be handled by monitor if moved_from moved_to events were captured. 
# it will use an tsv storing all the path inode number and hash to detect move.
ChangeEvent = namedtuple('ChangeEvent', ['monotonic_time', 'is_dir', 'event', 'path','moved_from'])


In [12]:
from collections import OrderedDict
cookieDic = OrderedDict()
COOKIE_DICT_MAX_SIZE = 16384

In [13]:
from collections import deque
to_process = deque()

In [14]:
import time
import multiCMD

In [95]:
import threading
GREEN_LIGHT = threading.Event()
GREEN_LIGHT.set()

In [16]:
import Tee_Logger
tl = Tee_Logger.teeLogger(systemLogFileDir='/dev/null', programName='reverberator', compressLogAfterMonths=0, deleteLogAfterYears=0, suppressPrintout=False,noLog=True)

In [17]:
def get_all_folders(path):
    folders = []
    try:
        with os.scandir(path) as entries:
            for entry in entries:
                if entry.is_dir(follow_symlinks=False):
                    folders.append(entry.path)
                    folders.extend(get_all_folders(entry.path))  # Recursive call
    except PermissionError:
        pass  # Skip folders without permission
    return folders

In [18]:
for i in range(10):
	folders = get_all_folders(os.path.expanduser('.'))
print(len(folders))
#folders

28


In [19]:
rtn = multiCMD.run_command(['sysctl','fs.inotify.max_queued_events'],timeout=5,quiet=True)

In [ ]:
resource.getrlimit(resource.RLIMIT_NOFILE)[0]

524288

In [21]:
test = {'1':'test'}
test.popitem()

('1', 'test')

In [97]:
# security limits
class inotify_resource_manager:
	def __init__(self):
		self.current_user_instances = 0
		self.current_user_watches = 0
		self.can_set_max_queued_events = True
		self.can_set_max_user_instances = True
		self.can_set_max_user_watches = True
		self.can_set_max_no_files = True
		self.can_set_max_no_files_user = True
		self.debug = False
		self.max_queued_events = self.getSysctlValues('fs.inotify.max_queued_events')
		self.max_user_instances = self.getSysctlValues('fs.inotify.max_user_instances')
		self.max_user_watches = self.getSysctlValues('fs.inotify.max_user_watches')
		self.max_no_files = self.getSysctlValues('fs.file-max')
		try:
			soft , hard = resource.getrlimit(resource.RLIMIT_NOFILE)
			if soft < hard:
				resource.setrlimit(resource.RLIMIT_NOFILE, (hard, hard))
			self.max_no_files_user = hard
		except:
			self.max_no_files_user = 0

	def __iter__(self):
		for key in self.__dict__.keys():
			yield key, getattr(self,key)

	def __str__(self):
		# do not include the can_set in the string representation
		return str(dict((k, v) for k, v in self.__dict__.items() if not k.startswith('can_set_')))
	
	def __repr__(self):
		return str(dict(self))

	def getSysctlValues(self,field):
		try:
			rtn = multiCMD.run_command(['sysctl',field],timeout=1,quiet=not self.debug)
			if rtn and rtn[0]:
				return int(rtn[0].split('=')[-1].strip())
		except:
			import traceback
			tl.error(traceback.format_exc())
			tl.error(f'Failed to get {field} value')
		return 0
	
	def setSysctlValues(self,field,value,value_name):
		try:
			if not getattr(self,f'can_set_{value_name}'):
				return False
			if value is ...:
				value = getattr(self,value_name)
			# check if the current value is less
			rtn = self.getSysctlValues(field)
			if rtn and rtn > value:
				tl.info(f'Current {field} is already higher than {value}')
				setattr(self,value_name,rtn)
				return False
			rc = multiCMD.run_command(['sudo','sysctl',f'{field}={value}'],timeout=1,quiet=not self.debug,return_code_only=True)
			if self.debug:
				tl.teeprint(f'sudo sysctl {field}={value} return code = {rc}')
			if rc:
				if self.debug:
					tl.teeprint(f'Failed to set {field} to {value}')
				else:
					tl.info(f'Failed to set {field} to {value}')
				setattr(self,f'can_set_{value_name}',False)
				if rtn:
					setattr(self,value_name,rtn)
				return False
			else:
				tl.teeok(f'Increased inotify {field} to {value}')
				setattr(self,value_name,value)
				return True
		except:
			try:
				import traceback
				tl.error(traceback.format_exc())
				tl.teeerror(f'Failed to set {field} = {value} for {value_name}')
				setattr(self,f'can_set_{value_name}',False)
				rtn = self.getSysctlValues(field)
				if rtn:
					setattr(self,value_name,rtn)
				else:
					setattr(self,value_name,-1)
			except:
				tl.teeerror(f'Failed to set {field} value')
		return False

	def increaseInotifyMaxQueueEvents(self,max_events:int = ...):
		if not self.can_set_max_queued_events:
			return False
		return self.setSysctlValues('fs.inotify.max_queued_events',max_events,'max_queued_events')

	def increaseInotifyMaxUserInstances(self,max_instances:int = ...):
		if not self.can_set_max_user_instances:
			return False
		return self.setSysctlValues('fs.inotify.max_user_instances',max_instances,'max_user_instances')

	def increaseInotifyMaxUserWatches(self,max_watches:int = ...):
		if not self.can_set_max_user_watches:
			return False
		rtn = self.setSysctlValues('fs.inotify.max_user_watches',max_watches,'max_user_watches')
		rtn2 = self.increaseNoFiles(max_watches)
		return rtn and rtn2

	def increaseNoFiles(self,nofile:int = ...):
		if not self.can_set_max_no_files_user:
			return False
		if nofile is ...:
			nofile = self.max_no_files
		# attempt to increase the open file limit
		try:
			# check if the current value is less
			if resource.getrlimit(resource.RLIMIT_NOFILE)[0] > nofile:
				if self.debug:
					tl.teeprint(f'Current open file limit is already higher than {nofile}')
				else:
					tl.info(f'Current open file limit is already higher than {nofile}')
				self.max_no_files = resource.getrlimit(resource.RLIMIT_NOFILE)[0]
				return False
			rtn = self.__increaseSystemNoFiles(nofile)
			resource.setrlimit(resource.RLIMIT_NOFILE, (nofile, nofile))
			tl.teeok(f'Increased open file limit to {nofile}')
			self.max_no_files = nofile
			return rtn
		except:
			import traceback
			tl.error(traceback.format_exc())
			tl.teeerror(f'Failed to increase open file limit to {nofile}')
			self.can_set_max_no_files_user = False
			return False

	def __increaseSystemNoFiles(self,nofile:int):
		if not self.can_set_max_no_files:
			return False
		return self.setSysctlValues('fs.file-max',nofile,'max_no_files')

	def increaseUserInstances(self,count:int = 1):
		self.current_user_instances += count
		if self.max_user_instances <= self.current_user_instances:
			self.increaseInotifyMaxUserInstances(self.max_user_instances * 2)
		return self.current_user_instances
	
	def decreaseUserInstances(self,count:int = 1):
		self.current_user_instances -= count
		return self.current_user_instances
	
	def increaseUserWatches(self,count:int = 1):
		self.current_user_watches += count
		if self.max_user_watches <= self.current_user_watches:
			self.increaseInotifyMaxUserWatches(self.max_user_watches * 2)
		return self.current_user_watches
	
	def decreaseUserWatches(self,count:int = 1):
		self.current_user_watches -= count
		return self.current_user_watches


In [67]:
irm = inotify_resource_manager()
irm.debug = True

In [70]:
dict(irm)

{'current_user_instances': 0,
 'current_user_watches': 0,
 'can_set_max_queued_events': True,
 'can_set_max_user_instances': True,
 'can_set_max_user_watches': True,
 'can_set_max_no_files': True,
 'can_set_max_no_files_user': False,
 'debug': True,
 'max_queued_events': 16384,
 'max_user_instances': 128,
 'max_user_watches': 1048576,
 'max_no_files': 9223372036854775807,
 'max_no_files_user': 524288}

In [71]:
irm.increaseNoFiles(524288 * 2)

False

In [79]:
def initializeFolderWatchers(inotify_obj:inotify_simple.INotify,monitor_path:str,wdDic:dict,irm:inotify_resource_manager,watch_flags:int):
	irm.increaseUserWatches()
	parentWatchDescriptor = inotify_obj.add_watch(monitor_path, watch_flags)
	wdDic[parentWatchDescriptor] = monitor_path
	allFolders = get_all_folders(monitor_path)
	irm.increaseUserWatches(len(allFolders))
	for folder in allFolders:
		wdDic[inotify_obj.add_watch(folder, watch_flags)] = folder
	return parentWatchDescriptor

In [73]:
# X		ACCESS = 0x00000001  #: File was accessed
# Y	MODIFY = 0x00000002  #: File was modified
# Y	ATTRIB = 0x00000004  #: Metadata changed
# X		CLOSE_WRITE = 0x00000008  #: Writable file was closed
# X		CLOSE_NOWRITE = 0x00000010  #: Unwritable file closed
# X		OPEN = 0x00000020  #: File was opened
# Y	MOVED_FROM = 0x00000040  #: File was moved from X
# Y	MOVED_TO = 0x00000080  #: File was moved to Y
# Y	CREATE = 0x00000100  #: Subfile was created
# Y	DELETE = 0x00000200  #: Subfile was deleted
# Y	DELETE_SELF = 0x00000400  #: Self was deleted
# Y	MOVE_SELF = 0x00000800  #: Self was moved

# Y	UNMOUNT = 0x00002000  #: Backing fs was unmounted
# Y	Q_OVERFLOW = 0x00004000  #: Event queue overflowed
# X		IGNORED = 0x00008000  #: File was ignored

# X		ONLYDIR = 0x01000000  #: only watch the path if it is a directory
# X		DONT_FOLLOW = 0x02000000  #: don't follow a sym link
# X		EXCL_UNLINK = 0x04000000  #: exclude events on unlinked objects
# X		MASK_ADD = 0x20000000  #: add to the mask of an already existing watch
# Y	ISDIR = 0x40000000  #: event occurred against dir
# X		ONESHOT = 0x80000000  #: only send event once

In [74]:
to_process = deque()
to_process_flag = threading.Event()
monitor_path = '/tmp/inotify_test'

In [137]:
def watchFolder(monitor_path:str,to_process:deque,to_process_flag:threading.Event,irm:inotify_resource_manager):
	global COOKIE_DICT_MAX_SIZE
	global tl
	flags = inotify_simple.flags
	watch_flags = flags.MODIFY | flags.ATTRIB | flags.MOVED_FROM | flags.MOVED_TO | flags.CREATE | flags.DELETE | flags.DELETE_SELF | flags.MOVE_SELF | flags.UNMOUNT | flags.Q_OVERFLOW
	selfDestructMask = flags.UNMOUNT | flags.DELETE_SELF | flags.MOVE_SELF
	cookieDic = OrderedDict()
	wdDic = {}
	irm.increaseUserInstances()
	with inotify_simple.INotify() as inotify_obj:
		mainWatchDescriptor = initializeFolderWatchers(inotify_obj,monitor_path,wdDic,irm,watch_flags)
		while GREEN_LIGHT.is_set():
			events =  inotify_obj.read(read_delay=1)
			monTime = time.monotonic_ns()
			for event in events:
				if event.mask & flags.Q_OVERFLOW:
					tl.teeerror('Queue overflow, event maybe lost')
					tl.teeprint('Attempting to increase inotify max_queued_events')
					irm.increaseInotifyMaxQueueEvents(irm.max_queued_events * 2)
					continue
				if event.mask & selfDestructMask:
					if event.wd == mainWatchDescriptor:
						# if UNMOUNT or IGNORED or DELETE_SELF, MOVE_SELF, return false
						irm.decreaseUserWatches(len(wdDic))
						irm.decreaseUserInstances()
						return False
					elif event.mask & flags.DELETE_SELF:
						wdDic.pop(event.wd)
						irm.decreaseUserWatches()
						continue
					elif event.mask & flags.UNMOUNT:
						wdDic.pop(event.wd)
						wdDic[inotify_obj.add_watch(wdDic[event.wd], watch_flags)] = wdDic[event.wd]
						continue
				if not event.wd in wdDic:
					tl.teeerror('Watch descriptor not found in wdDic, ignoring event')
					continue
				if not event.name:
					continue
				eventPath = os.path.join(wdDic[event.wd],event.name)
				isDir = True if event.mask & flags.ISDIR else False
				if event.mask & flags.CREATE:
					if isDir:
						initializeFolderWatchers(inotify_obj,eventPath,wdDic,irm,watch_flags)
					to_process.append(ChangeEvent(monTime,isDir,'create',eventPath,None))
				elif event.mask & flags.DELETE:
					if isDir:
						inotify_obj.rm_watch(event.wd)
						del wdDic[event.wd]
						irm.decreaseUserWatches()
					to_process.append(ChangeEvent(monTime,isDir,'delete',eventPath,None))
				elif event.mask & flags.ATTRIB:
					to_process.append(ChangeEvent(monTime,isDir,'attrib',eventPath,None))
				elif event.mask & flags.MOVED_FROM:
					cookieDic[event.cookie] = (event.wd,eventPath,isDir)
					while len(cookieDic) > COOKIE_DICT_MAX_SIZE:
						cookie,value = cookieDic.popitem(last=False)
						# treat the move as a delete
						if value[2]:
							inotify_obj.rm_watch(value[0])
							del wdDic[value[0]]
							irm.decreaseUserWatches()
						to_process.append(ChangeEvent(monTime,isDir,'delete',value[1],None))
					to_process.append(ChangeEvent(monTime,isDir,'move',eventPath,None))
				elif event.mask & flags.MOVED_TO:
					if event.cookie in cookieDic:
						to_process.append(ChangeEvent(monTime,isDir,'move',eventPath,cookieDic[event.cookie][1]))
						cookieDic.pop(event.cookie)
					else:
						if isDir:
							initializeFolderWatchers(inotify_obj,eventPath,wdDic,irm,watch_flags)
						to_process.append(ChangeEvent(monTime,isDir,'create',eventPath,None))
				elif event.mask & flags.MODIFY:
					to_process.append(ChangeEvent(monTime,isDir,'modify',eventPath,None))
				# else:
				# 	tl.info(f'Unprocessed event {event}')
			to_process_flag.set()


In [ ]:
#watchFolder(monitor_path,to_process,to_process_flag,irm)

In [143]:
watchFolderThread = threading.Thread(target=watchFolder,args=(monitor_path,to_process,to_process_flag,irm),daemon=True)
watchFolderThread.start()

In [ ]:
def testWatcherOperation(testCMD:list,to_process:deque,to_process_flag:threading.Event):
	multiCMD.run_command(testCMD)
	to_process_flag.wait(1)
	print(to_process)
	to_process.clear()
	to_process_flag.clear()


In [144]:
testWatcherOperation(['mkdir','/tmp/inotify_test/foo'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','echo hello > /tmp/inotify_test/test.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','echo world >> /tmp/inotify_test/test.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','mv /tmp/inotify_test/test.txt /tmp/inotify_test/test2.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','mv /tmp/inotify_test/test2.txt /tmp/outside.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','echo hello > /tmp/outside2.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','mv /tmp/outside2.txt /tmp/inotify_test/test2.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','mv /tmp/inotify_test/test2.txt /tmp/outside3.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','mv /tmp/outside3.txt /tmp/inotify_test/test3.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','rm /tmp/inotify_test/test3.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','echo hi >> /tmp/inotify_test/foo/test_rec.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','mv /tmp/inotify_test/foo /tmp/inotify_test/bar'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','touch /tmp/inotify_test/bar/test.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','echo hello > /tmp/inotify_test/bar/test.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','mv /tmp/inotify_test/bar/test.txt /tmp/inotify_test/bar/test2.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','mv /tmp/inotify_test/bar/test2.txt /tmp/inotify_test/test2_move_out.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','sudo mount -t tmpfs tmpfs /tmp/inotify_test/bar'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','touch /tmp/inotify_test/bar/test_temp.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','sudo umount /tmp/inotify_test/bar'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','touch /tmp/inotify_test/bar/test_temp.txt'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','touch /tmp/inotify_test/bar'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','mkdir -p /tmp/inotify_test/test/testrecursive/'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','touch /tmp/inotify_test/test/testrecursive/test.t'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','mv /tmp/inotify_test /tmp/inotify_test_2'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','rm -rf /tmp/inotify_test_2'],to_process,to_process_flag)
testWatcherOperation(['bash','-c','mkdir /tmp/inotify_test'],to_process,to_process_flag)

Running command: mkdir /tmp/inotify_test/foo
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Process exited with return code 0
deque([ChangeEvent(monotonic_time=36841419558418, is_dir=True, event='create', path='/tmp/inotify_test/foo', moved_from=None)])
Running command: bash -c echo hello > /tmp/inotify_test/test.txt
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Process exited with return code 0
deque([ChangeEvent(monotonic_time=36841423653022, is_dir=False, event='create', path='/tmp/inotify_test/test.txt', moved_from=None), ChangeEvent(monotonic_time=36841423653022, is_dir=False, event='modify', path='/tmp/inotify_test/test.txt', moved_from=None)])
Running command: bash -c ech

In [147]:
watchFolderThread.is_alive()

False

In [145]:
to_process

deque([])

In [146]:
to_process_flag

<threading.Event at 0x7f5e11a47f50: unset>